In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import json
import keras

In [ ]:
print(os.listdir("../input"))

In [ ]:
train_data = pd.read_csv('../input/train.csv')

In [ ]:
train_data.shape

In [ ]:
train_data.head()

In [ ]:
print('Number of unique heights = {0}'.format(train_data.Height.unique().shape[0]))
print('Minimum height = {0}'.format(train_data.Height.min()))
print('Maximum height = {0}'.format(train_data.Height.max()))
print('Average height = {0}'.format(train_data.Height.mean()))

In [ ]:
print('Number of unique widths = {0}'.format(train_data.Width.unique().shape[0]))
print('Minimum width = {0}'.format(train_data.Width.min()))
print('Maximum width = {0}'.format(train_data.Width.max()))
print('Average width = {0}'.format(train_data.Width.mean()))

In [ ]:
plt.hist(train_data.groupby(['ImageId']).agg('mean')['Height'], bins=100)
plt.xlabel('Height of images')
plt.ylabel('Number of images')
plt.show()

In [ ]:
plt.hist(train_data.groupby(['ImageId']).agg('mean')['Width'], bins=100)
plt.xlabel('Width of images')
plt.ylabel('Number of images')
plt.show()

In [ ]:
plt.hist(train_data.groupby(['ImageId']).agg('mean')['Height']/train_data.groupby(['ImageId']).agg('mean')['Width'], bins=20)
plt.xlabel('Ratio (height / width)')
plt.ylabel('Number of images')
plt.show()

Most images have height > width. <br>
The average height = 2236 <br>
The average width = 1762 <br>
We may consider this when resizing the image if we will resize them to the same size. <br>

In [ ]:
train_data.ImageId.unique().shape[0]

In [ ]:
num_classes = train_data.groupby(by=['ImageId']).count()['ClassId'].tolist()

In [ ]:
print('Min number of classes is {0} and maximum number is {1}'.format(min(num_classes), max(num_classes)))

In [ ]:
plt.hist(num_classes, bins=74)
plt.xlabel('Number of classes for one image')
plt.ylabel('Number of images')
plt.show()

In [ ]:
train_data.ClassId.unique().shape[0]

In [ ]:
with open('../input/label_descriptions.json') as json_file:  
    labels_desc = json.load(json_file)

In [ ]:
labels_desc.keys()

In [ ]:
labels_desc['info']

In [ ]:
categories = pd.DataFrame(labels_desc['categories'])

In [ ]:
attributes = pd.DataFrame(labels_desc['attributes'])

In [ ]:
categories.shape

In [ ]:
attributes.shape

In [94]:
categories

,id,level,name,supercategory
0,0,2,"shirt, blouse",upperbody
1,1,2,"top, t-shirt, sweatshirt",upperbody
2,2,2,sweater,upperbody
3,3,2,cardigan,upperbody
4,4,2,jacket,upperbody
5,5,2,vest,upperbody
6,6,2,pants,lowerbody
7,7,2,shorts,lowerbody
8,8,2,skirt,lowerbody
9,9,2,coat,wholebody


In [95]:
attributes

,id,level,name,supercategory
0,0,1,above-the-hip (length),length
1,1,1,hip (length),length
2,2,1,micro (length),length
3,3,1,mini (length),length
4,4,1,above-the-knee (length),length
5,5,1,knee (length),length
6,6,1,below the knee (length),length
7,7,1,midi,length
8,8,1,maxi (length),length
9,9,1,floor (length),length
